# Test-09
Ejecutado en **Kaggle**.

## Descripción
Se realiza **transfer learning** usando la **EfficientNetB3**.

# Configuración general

In [1]:
import pandas as pd

In [2]:
import tensorflow as tf

In [3]:
!pip install cloud_tpu_client

In [4]:
from cloud_tpu_client import Client
c = Client()
c.configure_tpu_version(tf.__version__, restart_type="always")

In [5]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
    print("Running on TPU ", tpu.cluster_spec().as_dict()["worker"])
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.TPUStrategy(tpu)
except ValueError:
    print("Not connected to a TPU runtime. Using CPU/GPU strategy")
    strategy = tf.distribute.MirroredStrategy()

Running on TPU  ['10.0.0.2:8470']


# Cargando el conjunto de datos

In [6]:
import numpy as np

In [7]:
import os

In [8]:
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/cifar100-nn-competition/y_train.npy
/kaggle/input/cifar100-nn-competition/x_test.npy
/kaggle/input/cifar100-nn-competition/x_train.npy


In [9]:
x_train_valid = np.load('/kaggle/input/cifar100-nn-competition/x_train.npy')
y_train_valid = np.load('/kaggle/input/cifar100-nn-competition/y_train.npy')
x_test = np.load('/kaggle/input/cifar100-nn-competition/x_test.npy')

# Separando el conjunto de datos

In [10]:
from sklearn.model_selection import train_test_split

In [11]:
x_train, x_valid, y_train, y_valid = train_test_split(x_train_valid, y_train_valid, test_size=0.2, random_state=20, stratify=y_train_valid)

# Escalando las imágenes

In [12]:
def resize_images(images, height, width):
  """ Resizes images using the bicubic interpolation method and returns them in a
      [0, 255] value mode.
      @param images Group of images
      @param height Height of the output image
      @param width Width of the output image
  """
  resized = tf.image.resize(images / 255, (width, height), method=tf.image.ResizeMethod.BICUBIC) * 255
  resized = tf.cast(resized, tf.int32)
  return resized

In [13]:
def resize_dataset(x_train, x_valid, x_test, height, width):
    """ Resizes the images of all train, valid and test dataset
        @param x_train
        @param x_valid
        @param x_test
        @param height
        @param width
    """
    return resize_images(x_train, height, width), resize_images(x_valid, height, width), resize_images(x_test, height, width)

# Modelo

In [14]:
from tensorflow.keras.applications import EfficientNetB0, EfficientNetB1, EfficientNetB2, EfficientNetB3

In [15]:
from tensorflow.keras.layers import (GlobalAveragePooling2D, Dense, Activation, 
                                     BatchNormalization, Reshape, Dropout, InputLayer,
                                     Flatten, Input, GlobalMaxPooling2D)

In [16]:
from tensorflow.keras.layers.experimental.preprocessing import (Resizing,
                                                                RandomFlip,
                                                                RandomZoom,
                                                                RandomRotation,
                                                                RandomTranslation)

In [17]:
from tensorflow.keras.models import Sequential, Model

In [18]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

In [19]:
from tensorflow.keras.optimizers import Adam

In [20]:
from tensorflow import keras

In [21]:
import pandas as pd

In [22]:
def generate_submission(predictions, filepath='submission.csv'):
    """ Generate the .csv file to submit in the challenge
        @param predictions Predictions made by the model from the test dataset
        @param filepath Filepath for the file generated
    """
    df = pd.DataFrame(predictions, columns=['label'])
    df.index.name = 'Id'
    df.to_csv(filepath)

### Preprocesando imágenes

In [23]:
x_train_tf, x_valid_tf, x_test_tf = resize_dataset(x_train, x_valid, x_test, 300, 300)

### Creación del modelo

In [26]:
with strategy.scope():
    # Create the model
    model = Sequential()
    model.add(InputLayer(input_shape=(300, 300, 3)))
    model.add(EfficientNetB3(include_top=False, drop_connect_rate=0.4, weights='imagenet', input_shape=(300, 300, 3)))
    model.add(GlobalAveragePooling2D())
    model.add(BatchNormalization())
    model.add(Dropout(0.6)) 
    model.add(Dense(units=2048))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Activation('relu'))
    model.add(Dense(units=100))
    model.add(Activation('softmax'))
    
    # Freeze EffNetB3 Weights
    model.layers[0].trainable = True
    
    # Compile the model
    model.compile(loss='sparse_categorical_crossentropy',
                optimizer=Adam(learning_rate=0.0001), # ¿Podrias bajar esto o subirlo?
                metrics=['accuracy'],
                steps_per_execution=30
              )

# Summarize
model.summary()

43941888/43941136 [==============================] - 0s 0us/step
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
efficientnetb3 (Functional)  (None, 10, 10, 1536)      10783535  
_________________________________________________________________
global_average_pooling2d (Gl (None, 1536)              0         
_________________________________________________________________
batch_normalization (BatchNo (None, 1536)              6144      
_________________________________________________________________
dropout (Dropout)            (None, 1536)              0         
_________________________________________________________________
dense (Dense)                (None, 2048)              3147776   
_________________________________________________________________
batch_normalization_1 (Batch (None, 2048)              8192      
_________________________________________________________

In [27]:
# Create the ModelCheckpoint callback to save the best model during training
mc_callback = ModelCheckpoint('model_1.hdf5',
                              monitor='val_accuracy',
                              save_best_only=True,
                              verbose=0,
                              mode='max'
                             )

# Create the EarlyStopping callback to stop when not improving during training
es_callback = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)

# Create the ReduceLROnPlateau callback to reduce the learning rate when not improving
lr_callback = ReduceLROnPlateau(monitor='val_accuracy', factor=0.2, patience=2, min_lr=0.001)

# Train the model
epochs = 100
batch_size = 256
model.fit(x_train_tf, y_train,
          validation_data=(x_valid_tf, y_valid), 
          callbacks=[mc_callback, lr_callback],
          epochs=epochs,
          batch_size=batch_size
          )

Epoch 1/100
157/157 [==============================] - 213s 1s/step - loss: 4.6211 - accuracy: 0.0691 - val_loss: 3.4211 - val_accuracy: 0.3633
Epoch 2/100
157/157 [==============================] - 33s 208ms/step - loss: 2.0785 - accuracy: 0.4587 - val_loss: 1.3316 - val_accuracy: 0.6799
Epoch 3/100
157/157 [==============================] - 33s 208ms/step - loss: 1.3458 - accuracy: 0.6205 - val_loss: 0.7063 - val_accuracy: 0.7914
Epoch 4/100
157/157 [==============================] - 33s 209ms/step - loss: 1.0154 - accuracy: 0.7051 - val_loss: 0.5867 - val_accuracy: 0.8188
Epoch 5/100
157/157 [==============================] - 33s 208ms/step - loss: 0.8493 - accuracy: 0.7510 - val_loss: 0.5452 - val_accuracy: 0.8332
Epoch 6/100
157/157 [==============================] - 33s 208ms/step - loss: 0.7206 - accuracy: 0.7846 - val_loss: 0.5111 - val_accuracy: 0.8437
Epoch 7/100
157/157 [==============================] - 33s 209ms/step - loss: 0.6356 - accuracy: 0.8083 - val_loss: 0.4899 - v

KeyboardInterrupt: 

### Evaluación del modelo

In [28]:
# Load the model and show the final metrics
model = keras.models.load_model('model_1.hdf5')

# Train and validation metrics
_, train_acc = model.evaluate(x_train_tf, y_train, verbose=0)
_, valid_acc = model.evaluate(x_valid_tf, y_valid, verbose=0)

# Show result
print(f'[Accuracy] Train: {round(train_acc, 3)} Valid: {round(valid_acc, 3)}')

[Accuracy] Train: 1.0 Valid: 0.876


In [29]:
y_pred = model.predict(x_test_tf).argmax(axis=-1)

In [31]:
generate_submission(y_pred, filepath='submission1.csv')